### Q1&Q2 Upload data and load into MySQL

Used PSFTP.

Used commands recommended.

Used DATE format for dates, and other formats as appropriate.

### Q3&Q4 Observe and key all tables

First use id column as primary key for user, subscription and tables:

```
ALTER TABLE subscriptions ADD PRIMARY KEY (id);

ALTER TABLE users ADD PRIMARY KEY (id);

ALTER TABLE purchases ADD PRIMARY KEY (id);
```

And make necessary columns NOT NULL e.g.:

```
ALTER TABLE assignments MODIFY sub_id INT NOT NULL;
```

FOREIGN KEY on subscriptions.id:

```
ALTER TABLE `assignments` 
ADD FOREIGN KEY(`sub_id`) 
REFERENCES subscriptions(id);
```
FOREIGN KEY on users.id:

```
ALTER TABLE `purchases` 
ADD FOREIGN KEY(`user_id`) 
REFERENCES users(id);	

ALTER TABLE `subscriptions` 
ADD FOREIGN KEY(`user_id`) 
REFERENCES users(id);	
```
Join works much better now.

### Q5 What's wrong with data set?

<strong>Purchases can not be uniquely matched to subscriptions, since users may have multiple subscriptions.</strong>

Some other possible issues:

There are ~3x subscriptions starting in november & december compared to other months.

There are 2 duplicate IP addresses.

Most of the names are duplicate/triplicate.

Twice the purchase amount in January compared to any other month.

All the purchases are roughly the same value, wouldn't make sense for a Costco-like store.

### Q6 Evaluate AB Test

First, let's count up the total number of subscriptions that are active in the test period.

We need to select accounts that ended AFTER 2016-07-01, or are still active, end-date = NULL.

```
select test_group, count(*) from 
(
select s.id, test_group from subscriptions s 
join assignments 
on s.id=sub_id 
where ((end_date > '2016-07-01') or (end_date IS NULL))
) as tab
group by test_group;

+------------+----------+
| test_group | count(*) |
+------------+----------+
| control    |    17439 |
| test       |    24933 |
+------------+----------+
2 rows in set (0.00 sec)

```
This seems a little unbalanced, so it appears that the test group and control group are not assigned completely at random. Let's see if there is any counfounding by the number of accounts started after the start date:

```
select test_group, count(*) from 
(
select s.id, test_group from subscriptions s 
join assignments 
on s.id=sub_id 
where ((end_date > '2016-07-01') or (end_date IS NULL))
and (start_date > '2016-07-01')
) as tab
group by test_group;

+------------+----------+
| test_group | count(*) |
+------------+----------+
| control    |     6225 |
| test       |     6432 |
+------------+----------+
2 rows in set (0.15 sec)

```

So the fraction of new users in control is 6225/17439 = 0.3569

And test, 6432/24933 = 0.2579

So... this seems quite skewed. We should restrict the test to only accounts started after the start date.

Now let's address another possible problem. Purchases cannot be uniquely assigned to a subscription. Let's see if any of the users are assigned to duplicate user account

```
select user_id, count(*) from (
select s.id,user_id, test_group 
from subscriptions s 
join assignments 
on 
s.id=sub_id 
where ((end_date > '2016-07-01') or (end_date IS NULL))
and (start_date < '2016-07-01')
) as tab group by user_id having count(*) > 1;

+---------+----------+
| user_id | count(*) |
+---------+----------+
|   57891 |        2 |
|   89857 |        2 |
|  110382 |        2 |
|  121530 |        2 |
|  134665 |        2 |
|  135397 |        2 |
|  136783 |        2 |
|  137577 |        2 |
|  140524 |        2 |
+---------+----------+
9 rows in set (0.00 sec)

```

So there are 9 users (18 subscriptions) that may double-count purchases.

Let's see what the total value of purchases are here to see if this may somehow strongly skew the results.

```
select tabb.user_id, sum(amount) as amount_per_user from
(
    select user_id, count(*) from (
    select s.id,user_id, test_group 
    from subscriptions s 
    join assignments 
    on 
    s.id=sub_id 
    where ((end_date > '2016-07-01') or (end_date IS NULL))
    and (start_date < '2016-07-01')
    ) as tab group by user_id having count(*) > 1
) as tabb 
join purchases
on tabb.user_id = purchases.user_id
group by user_id;

Empty set (0.17 sec)

```

So none of these double account users actually made purchases, so there's no need to actually take them out.

Now let's finally count up the summary statistics, average and standard dev of purchase amount per subscriber.

Something to note: we need to restrict user accounts AND purchases by date.

```
select test_group, avg(tot_per_user) as avg, stddev(tot_per_user) as std from
(
    select user_id,sum(amount) as tot_per_user,test_group from
    (
        select s.id,s.user_id,IFNULL(amount, 0) as amount,test_group,purchases.date
        from subscriptions s 
        join assignments 
        on s.id=sub_id 
        left join purchases 
        on purchases.user_id = s.user_id 
        where ((end_date > '2016-07-01') or (end_date IS NULL))
        and ((purchases.date > '2016-07-01') or (purchases.date IS NULL))
        and (start_date < '2016-07-01')
    ) as sums
    group by user_id
) as final
group by test_group;

+------------+-----------+----------+
| test_group | avg       | std      |
+------------+-----------+----------+
| control    | 10.331976 | 9.812589 |
| test       |  6.261994 | 9.211428 |
+------------+-----------+----------+

```

WOW! Looks like this change SUCKS! Abort immediately. Let's just check without removing the user accounts started after the start date:

```

select test_group, avg(tot_per_user) as avg, stddev(tot_per_user) as std from
(
    select user_id,sum(amount) as tot_per_user,test_group from
    (
        select s.id,s.user_id,IFNULL(amount, 0) as amount,test_group,purchases.date
        from subscriptions s 
        join assignments 
        on s.id=sub_id 
        left join purchases 
        on purchases.user_id = s.user_id 
        where ((end_date > '2016-07-01') or (end_date IS NULL))
        and ((purchases.date > '2016-07-01') or (purchases.date IS NULL))
    ) as sums
    group by user_id
) as final
group by test_group;

+------------+-----------+-----------+
| test_group | avg       | std       |
+------------+-----------+-----------+
| control    | 12.636420 | 10.898481 |
| test       |  9.269383 | 11.154681 |
+------------+-----------+-----------+

```

Still SUCKS! OK now let's do the t-test of significance.

The test statistic will be:

$t_{n1+n2-2} = \dfrac { \overline{X_1} - \overline{X_2} } { \sigma \sqrt { \tfrac{1}{n_1} + \tfrac{1}{n_12} } } $

Assuming that the distributions have roughly equal variance (and they appear to).

Note, the underlying population distributions are definitely not normal, and appear to have right-skew, but the sample means should be approximately normal because we have high numbers of samples.

Let's get the actual number of units in each sample:

```
select test_group, count(*) from 
(
select s.id, test_group from subscriptions s 
join assignments 
on s.id=sub_id 
where ((end_date > '2016-07-01') or (end_date IS NULL))
and (start_date < '2016-07-01')
) as tab
group by test_group;

+------------+----------+
| test_group | count(*) |
+------------+----------+
| control    |    11117 |
| test       |    18395 |
+------------+----------+


```

In [8]:
from scipy.stats import t

tval = (10.33 - 6.26)/(9.5)/((1/11117.0 + 1/18395.0)**0.5)
print 't-statistic =', tval

print 'cumulative distribution = ', t.cdf(tval, 11117 + 18395, loc=0, scale=1)
p = 2* (1-t.cdf(tval, 11117 + 18395, loc=0, scale=1))
print 'p-val = ', p

t-statistic = 35.6627789552
cumulative distribution =  1.0
p-val =  0.0


Now let's do a power calculation. Let's just do a calculation on what kind of sample size we would need to detect the kind of difference we have with 80% power.

So in other words $P(t_{stat} > t_{crit} | \mu_1 - \mu_2 = 4.07) = 80\%$

For a 2 sided t-test, t-crit ~ 2 (depending on df, we can check this assumption later).

The true calculation here is:

$P(t_{df} > (2 - E[t_{stat}]) = 0.8$

Here we will simplify the t-distribution to a normal one, (we can check the validity of this later).

So we are looking for a value that a random normal will be greater than, 80% of the time.

This is just $CDF^{-1}(0.2) = -0.841621$

$2 - \dfrac{4.07}{9.5\sqrt{\tfrac{2}{n}}} = -0.841621$

Where n is the number in each sample (assuming equal).

$ n = ((2+0.84)*\sqrt2 * 9.5 / 4.07)^2 $

$(2.84*1.41*9.5/4.07)^2 = 87 $ so we expect to get 80% power with n~90, we have more than 10,000.

The assumption of normality holds up. At $df\sim2*90$, the t-distribution is nearly normal.


### Q7 Company evaluation

We can look at how sales are changing month to month:

```
select sum(amount) from purchases group by month(date);

+-------------+
| sum(amount) |
+-------------+
|   254272.05 |
|   164846.26 |
|   110667.99 |
|   104222.72 |
|   141337.98 |
|   141637.65 |
|   160638.66 |
|   166556.71 |
|     5380.47 |
+-------------+

```

There's a steep dropoff after January, (after-holiday sales?) but otherwise it looks mostly stable-ish. 

Also, we can count number of subscriptions beginning (and that have ended) in each year:

```
select year(start_date), count(*) as total, count(end_date) as churned from subscriptions group by year(start_date); 


+------------------+-------+---------+
| year(start_date) | total | churned |
+------------------+-------+---------+
|             2012 |  3870 |    3828 |
|             2013 | 10815 |   10564 |
|             2014 | 31351 |   30121 |
|             2015 | 62568 |   56017 |
|             2016 | 45684 |   22751 |
+------------------+-------+---------+

```

So it looks like overall there is pretty healthy growth in new subscriptions overall, but the churn seems high... only ~10% of subscriptions starting in 2015 are still current. About 50% of subscriptions this year have already churned.

### Q8 Active user table

First create a date table with all dates from:

1/9/2012 - 9/1/2016
```
DROP PROCEDURE IF EXISTS filldates;
DELIMITER |
CREATE PROCEDURE filldates(dateStart DATE, dateEnd DATE)
BEGIN
  WHILE dateStart <= dateEnd DO
    INSERT INTO _date (datelist) VALUES (dateStart);
    SET dateStart = date_add(dateStart, INTERVAL 1 DAY);
  END WHILE;
END;
|
DELIMITER ;
CALL filldates('2012-01-09','2016-09-01');

SELECT * FROM _date;

...
| 2016-08-27 |
| 2016-08-28 |
| 2016-08-29 |
| 2016-08-30 |
| 2016-08-31 |
| 2016-09-01 |
+------------+
```

Try to make the table

```
SELECT 
user_id, 
datelist as date, 
    (case 
    when end_date IS NULL
    then TRUE
    when (end_date IS NOT NULL) and (datelist<end_date)
    then TRUE
    else FALSE
    end) as is_active ,
start_date as signup_date,
signup_platform  
from subscriptions 
join _date 
on start_date<datelist 
join users 
on users.id = user_id
WHERE user_id BETWEEN 100000 AND 105000;

|  104997 | 2016-09-01 |         0 | 2015-12-25  | ios             |
|  104998 | 2016-09-01 |         0 | 2015-12-25  | ios             |
|  104999 | 2016-09-01 |         0 | 2015-12-25  | ios             |
|  105000 | 2016-09-01 |         0 | 2015-12-25  | ios             |
+---------+------------+-----------+-------------+-----------------+
1288429 rows in set (1.97 sec)

(FALSE evaluates to 0)

```

...and MySQL crashes if you don't restrict user_id

To make it more efficient you could only add today's entries. Entries from the past would not change.

Let's count churn by date.

```
SELECT end_date, count(end_date) as num_churn from subscriptions group by end_date;

+------------+-----------+
| end_date   | num_churn |
+------------+-----------+
...
...
...
| 2016-08-24 |       182 |
| 2016-08-25 |       178 |
| 2016-08-26 |       185 |
| 2016-08-27 |       187 |
| 2016-08-28 |       196 |
| 2016-08-29 |       182 |
| 2016-08-30 |       189 |
| 2016-08-31 |       185 |
| 2016-09-01 |       176 |
+------------+-----------+
```

### Q9 Annual vs monthly

We can look at the purchase amount this year, grouped by subscription type:

```
select sum(amount), sub_type 
from purchases 
join subscriptions 
on subscriptions.user_id = purchases.user_id 
group by sub_type;

+-------------+----------+
| sum(amount) | sub_type |
+-------------+----------+
|    91742.75 | annual   |
|  1178183.30 | monthly  |
+-------------+----------+


```

So the change should definitely be made. 

Expected revenue increase would be ~ 0.10 * (1,200,000 - 100,000) = 110,000.

Annualized this would be about 110,000/8*12 = 165,000